In [1]:
# Import dependencies
import pandas as pd
import numpy as np
from config import db_password
from sqlalchemy import create_engine
import psycopg2
import pandas.io.sql as sqlio
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
#create database connection variable 
conn = psycopg2.connect(user="postgres", password=db_password, host="localhost", database="lichess_data")

In [3]:
#execute query and save it to a variable
query="select * from chess_data"
chess_df = sqlio.read_sql_query(query,conn)

In [4]:
chess_df

,created_at,turns,winner,white_id,white_rating,black_id,black_rating,rating_difference,opening_eco,opening_name,moves
0,2017-08-31 20:06:40,13,white,bourgris,1500,a-00,1191,309,D10,Slav Defense: Exchange Variation,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4 Nc3 Ba5 Bf4
1,2017-08-30 21:53:20,16,black,a-00,1322,skinnerua,1261,61,B00,Nimzowitsch Defense: Kennedy Variation,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...
2,2017-08-30 21:53:20,61,white,ischia,1496,a-00,1500,-4,C20,King's Pawn Game: Leonardis Variation,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...
3,2017-08-30 16:20:00,61,white,daniamurashov,1439,adivanov2009,1454,-15,D02,Queen's Pawn Game: Zukertort Variation,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...
4,2017-08-29 18:06:40,95,white,nik221107,1523,adivanov2009,1469,54,C41,Philidor Defense,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...
...,...,...,...,...,...,...,...,...,...,...,...
1019624,2016-07-19 11:24:04,44,black,DGal,1207,DeLoreanGo,1374,-167,D30,Queen's Gambit Declined,c4 e6 d4 d5 c5 Nc6 Nf3 Nf6 Bg5 Be7 e3 e5 Nxe5 ...
1019625,2016-07-03 09:58:58,64,draw,MI22,1253,ruebezahl,1462,-209,A40,"Englund Gambit Declined, Reversed French",d4 e5 e3 exd4 Qxd4 Nc6 Qf4 Nf6 Nf3 Bd6 Qc4 O-O...
1019626,2016-07-14 17:04:13,59,white,igorium,1680,amarant,1535,145,C01,"French Defense: Exchange Variation, Svenonius ...",e4 e6 d4 d5 exd5 exd5 Nc3 Nf6 Bg5 Bb4 Qd3 O-O ...
1019627,2016-07-14 19:54:50,46,black,yusuf111907,1452,sullc84,1551,-99,B21,Sicilian Defense: Smith-Morra Gambit #2,e4 c5 d4 cxd4 Qxd4 Nf6 Nc3 e6 Bg5 Nc6 Qd3 Ne5 ...


In [8]:
# Split moves column into moves df
moves_df = chess_df['moves'].str.split(' ', n=10, expand=True)

# Drop column 10 and rename columns
moves_df=moves_df.drop(10,axis=1)
moves_df.columns= ["Wm1","Bm1","Wm2","Bm2","Wm3","Bm3","Wm4","Bm4","Wm5","Bm5"]

moves_df["outcome"] = chess_df["winner"]

# drop na
moves_df = moves_df.dropna()

In [9]:
# Changing moves to numbers
for col in moves_df.columns:
    
    # Get list of unique values
    values = list(set(moves_df[col].values))
    
    # Create numerical dictionary
    values_with_indexes = {}
    for i, v in enumerate(values):
        values_with_indexes[v] = i
    
    # Replace column
    moves_df.replace({col: values_with_indexes},inplace=True)

moves_df.head()

,Wm1,Bm1,Wm2,Bm2,Wm3,Bm3,Wm4,Bm4,Wm5,Bm5,outcome
0,16,11,37,142,277,43,330,17,473,544,3
1,16,16,17,84,106,237,278,492,564,165,2
2,11,17,59,129,291,82,271,7,230,559,3
3,16,11,23,42,224,1,290,369,420,370,3
4,11,17,23,129,195,203,445,8,3,562,3


In [10]:
# Export csv containing sample data to be imported into ML
moves_df.to_csv("ML_sample_data.csv")

In [ ]:
# Split our preprocessed data into our features and target arrays
y = moves_df["outcome"].values
X = moves_df.drop("outcome",1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Create the keras sequential model
number_input_features = len(X_train_scaled[0])
nn_model = tf.keras.models.Sequential()

In [ ]:
# Add the first layer including input layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid", input_dim = number_input_features))

In [ ]:
# Add the ouput layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [ ]:
# Create a summary to check the structure of the sequential model
nn_model.summary()

In [ ]:
# Compile the model and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=5)

In [ ]:
#  now that our deep learning model is properly trained, we can evaluate the model's performance by testing its predictive capabilities on our testing dataset
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")